# Storing and Retrieving Data

In [1]:
nobel_winners = [
    {'category': 'Physics',
    'name': 'Albert Einstein',
    'nationality': 'Swiss',
    'sex': 'male',
    'year': 1921},
    {'category': 'Physics',
    'name': 'Paul Dirac',
    'nationality': 'British',
    'sex': 'male',
    'year': 1933},
    {'category': 'Chemistry',
    'name': 'Marie Curie',
    'nationality': 'Polish',
    'sex': 'female',
    'year': 1911},
]

### Working with files manually (without csv module)

In [10]:
cols = list(nobel_winners[0].keys())
cols.sort()
with open('data/nobel_winners.csv', 'w') as f:
    f.write(','.join(cols) + '\n')
    for o in nobel_winners:
        row = [str(o[col]) for col in cols]
        f.write(','.join(row) + '\n')

### Now with csv

In [24]:
import csv

#Write
with open('data/nobel_winners.csv', 'wb') as f:
    fieldnames = nobel_winners[0].keys()
    fieldnames.sort()
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for w in nobel_winners:
        writer.writerow(w)
        
#Read using csv.reader which returns strings
with open('data/nobel_winners.csv') as f:
    reader = csv.reader(f)
    for row in reader:
        print(row)
        
#Read in more convenient python dict form
with open('data/nobel_winners.csv') as f:
    reader = csv.DictReader(f)
    nobel_winners = list(reader)
for w in nobel_winners:
    w['year'] = int(w['year']) #convert string to int
print(nobel_winners)



['category', 'name', 'nationality', 'sex', 'year']
['Physics', 'Albert Einstein', 'Swiss', 'male', '1921']
['Physics', 'Paul Dirac', 'British', 'male', '1933']
['Chemistry', 'Marie Curie', 'Polish', 'female', '1911']
[{'category': 'Physics', 'nationality': 'Swiss', 'year': 1921, 'name': 'Albert Einstein', 'sex': 'male'}, {'category': 'Physics', 'nationality': 'British', 'year': 1933, 'name': 'Paul Dirac', 'sex': 'male'}, {'category': 'Chemistry', 'nationality': 'Polish', 'year': 1911, 'name': 'Marie Curie', 'sex': 'female'}]


### Now save in JSON instead of csv

In [25]:
import json

with open('data/nobel_winners.json', 'w') as f:
    json.dump(nobel_winners, f)
    
with open('data/nobel_winners.json') as f:
    nobel_winners = json.load(f)
    #print(f.read()) #note that we don't need to convert year to ints
print(nobel_winners)

[{u'category': u'Physics', u'nationality': u'Swiss', u'sex': u'male', u'name': u'Albert Einstein', u'year': 1921}, {u'category': u'Physics', u'nationality': u'British', u'sex': u'male', u'name': u'Paul Dirac', u'year': 1933}, {u'category': u'Chemistry', u'nationality': u'Polish', u'sex': u'female', u'name': u'Marie Curie', u'year': 1911}]


### Dealing with Dates and Times


In [26]:
from datetime import datetime
#json.dumps(datetime.now()) # will trigger an error

In [27]:
import datetime
from dateutil import parser
import json

class JSONDateTimeEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (datetime.date, datetime.datetime)):
            return obj.isoformat()
        else:
            return json.JSONEncoder.default(self, obj)
        
def mydumps(obj):
    return json.dumps(obj, cls=JSONDateTimeEncoder)
now_str = mydumps({'time': datetime.datetime.now()})
now_str


'{"time": "2017-08-22T22:43:12.078080"}'

In [28]:
from datetime import datetime
import dateutil

d = datetime.now()
d_iso = d.isoformat()  # convert to text
print(d_iso)

d = dateutil.parser.parse(d_iso) # back to a datetime object
print(d)

2017-08-22T22:43:12.593377
2017-08-22 22:43:12.593377


# Databases

## SQL Databases (all using SQLAlchemy as go-between interface)

Using SQLAlchemy -- first with SQLite

Begin by creating the engine.

In [29]:
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Enum

engine = sqlalchemy.create_engine('sqlite:///data/nobel_prize.db', echo=True)

#Now define the database tables using the new declarative style (recommended)

Base = declarative_base()

# now use Base to define the tables

class Winner(Base):
    __tablename__ = 'winners'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    category = Column(String)
    year = Column(Integer)
    nationality = Column(String)
    sex = Column(Enum('male', 'female'))
    
    def __repr__(self):
        return("<Winner(name={}, category={}, year={})>".format(self.name,
                                    self.category, self.year))
# Having defined Base subclass, we supply is metadata.create_all method
# with our database engine to create our database.  Because we had set
# echo=True, it will tell us the SQL instructions it generates.
Base.metadata.create_all(engine)

2017-08-22 22:43:14,805 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-08-22 22:43:14,806 INFO sqlalchemy.engine.base.Engine ()
2017-08-22 22:43:14,807 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-08-22 22:43:14,807 INFO sqlalchemy.engine.base.Engine ()
2017-08-22 22:43:14,808 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("winners")
2017-08-22 22:43:14,809 INFO sqlalchemy.engine.base.Engine ()
2017-08-22 22:43:14,811 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE winners (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	category VARCHAR, 
	year INTEGER, 
	nationality VARCHAR, 
	sex VARCHAR(6), 
	PRIMARY KEY (id), 
	CHECK (sex IN ('male', 'female'))
)


2017-08-22 22:43:14,812 INFO sqlalchemy.engine.base.Engine ()
2017-08-22 22:43:14,819 INFO sqlalchemy.engine.base.Engine COMMIT


In [30]:
# Now start adding winner instances to this new database
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

albert = Winner(**nobel_winners[0])
session.add(albert)
session.new #new is the set of items queued up this session
# these aren't committed to the DB until we call .commit
session.expunge(albert) #take this out of the queue
winner_rows = [Winner(**w) for w in nobel_winners]
session.add_all(winner_rows)
session.commit()

2017-08-22 22:43:18,219 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-08-22 22:43:18,221 INFO sqlalchemy.engine.base.Engine INSERT INTO winners (name, category, year, nationality, sex) VALUES (?, ?, ?, ?, ?)
2017-08-22 22:43:18,221 INFO sqlalchemy.engine.base.Engine (u'Albert Einstein', u'Physics', 1921, u'Swiss', u'male')
2017-08-22 22:43:18,223 INFO sqlalchemy.engine.base.Engine INSERT INTO winners (name, category, year, nationality, sex) VALUES (?, ?, ?, ?, ?)
2017-08-22 22:43:18,223 INFO sqlalchemy.engine.base.Engine (u'Paul Dirac', u'Physics', 1933, u'British', u'male')
2017-08-22 22:43:18,226 INFO sqlalchemy.engine.base.Engine INSERT INTO winners (name, category, year, nationality, sex) VALUES (?, ?, ?, ?, ?)
2017-08-22 22:43:18,227 INFO sqlalchemy.engine.base.Engine (u'Marie Curie', u'Chemistry', 1911, u'Polish', u'female')
2017-08-22 22:43:18,230 INFO sqlalchemy.engine.base.Engine COMMIT


### Querying the Database

In [31]:
session.query(Winner).count()  # output: 3

result = session.query(Winner).filter_by(nationality='Swiss') #filter_by uses keyword expressions
list(result)

result = session.query(Winner).filter(Winner.category == 'Physics', #filter uses SQL expressions
                                     Winner.nationality != 'Swiss')
list(result)

2017-08-22 22:43:22,722 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-08-22 22:43:22,724 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT winners.id AS winners_id, winners.name AS winners_name, winners.category AS winners_category, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners) AS anon_1
2017-08-22 22:43:22,725 INFO sqlalchemy.engine.base.Engine ()
2017-08-22 22:43:22,727 INFO sqlalchemy.engine.base.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.category AS winners_category, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners 
WHERE winners.nationality = ?
2017-08-22 22:43:22,728 INFO sqlalchemy.engine.base.Engine ('Swiss',)
2017-08-22 22:43:22,730 INFO sqlalchemy.engine.base.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.category AS winners_category, winner

[<Winner(name=Paul Dirac, category=Physics, year=1933)>]

In [32]:
session.query(Winner).get(3) # get the 3rd record

res = session.query(Winner).order_by('year')
list(res)

2017-08-22 22:43:25,228 INFO sqlalchemy.engine.base.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.category AS winners_category, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners 
WHERE winners.id = ?
2017-08-22 22:43:25,229 INFO sqlalchemy.engine.base.Engine (3,)
2017-08-22 22:43:25,231 INFO sqlalchemy.engine.base.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.category AS winners_category, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners ORDER BY winners.year
2017-08-22 22:43:25,232 INFO sqlalchemy.engine.base.Engine ()


[<Winner(name=Marie Curie, category=Chemistry, year=1911)>,
 <Winner(name=Albert Einstein, category=Physics, year=1921)>,
 <Winner(name=Paul Dirac, category=Physics, year=1933)>]

### Reconstruct our original python dict from out of the DB

In [33]:
def inst_to_dict(inst, delete_id=True):
    dat = {}
    for column in inst.__table__.columns: #access instance's table class to get list of column objects
        dat[column.name] = getattr(inst, column.name)
    if delete_id:
        dat.pop('id')
    return dat

winner_rows = session.query(Winner)
new_nobel_winners = [inst_to_dict(w) for w in winner_rows]
new_nobel_winners

2017-08-22 22:43:30,425 INFO sqlalchemy.engine.base.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.category AS winners_category, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners
2017-08-22 22:43:30,426 INFO sqlalchemy.engine.base.Engine ()


[{'category': u'Physics',
  'name': u'Albert Einstein',
  'nationality': u'Swiss',
  'sex': u'male',
  'year': 1921},
 {'category': u'Physics',
  'name': u'Paul Dirac',
  'nationality': u'British',
  'sex': u'male',
  'year': 1933},
 {'category': u'Chemistry',
  'name': u'Marie Curie',
  'nationality': u'Polish',
  'sex': u'female',
  'year': 1911}]

### Updating info in the DB

In [34]:
marie = session.query(Winner).get(3)
marie.nationality = 'French'
session.dirty #shows pending changes

session.commit()

# Deleting a result of query
session.query(Winner).filter_by(name='Albert Einstein',).delete()

# Drop the whole table
session.close()
Winner.__table__.drop(engine)

2017-08-22 22:43:37,560 INFO sqlalchemy.engine.base.Engine UPDATE winners SET nationality=? WHERE winners.id = ?
2017-08-22 22:43:37,561 INFO sqlalchemy.engine.base.Engine ('French', 3)
2017-08-22 22:43:37,562 INFO sqlalchemy.engine.base.Engine COMMIT
2017-08-22 22:43:37,575 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-08-22 22:43:37,576 INFO sqlalchemy.engine.base.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.category AS winners_category, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners 
WHERE winners.id = ?
2017-08-22 22:43:37,577 INFO sqlalchemy.engine.base.Engine (1,)
2017-08-22 22:43:37,579 INFO sqlalchemy.engine.base.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.category AS winners_category, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners 
WHERE winners.id = ?
2017-08-22 22:

## NoSQL Databases (MongoDB)

Creating the database and collection

In [35]:
from pymongo import MongoClient

DB_NOBEL_PRIZE = 'nobel_prize' # use string constants or a spell error in retrieval will create new table.
COLL_WINNERS = 'winners' # winners collection

client = MongoClient()
db = client[DB_NOBEL_PRIZE] # creates (or retrieves if it exists) a database
coll = db[COLL_WINNERS] # creates (or retrieves if it exists) a collection

Accessing the data

In [40]:
from pymongo import MongoClient

def get_mongo_database(db_name, host='localhost', port=27017, username=None, password=None):
    '''Get named database from MongoDB with/out authentication'''
    if username and password:
        mongo_uri = 'mongodb://{}:{}@{}/{}'.format(username, password, host, db_name)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)
        
    return conn[db_name]

db = get_mongo_database(DB_NOBEL_PRIZE)
coll = db[COLL_WINNERS]

# Insert the dataset
coll.insert_many(nobel_winners)
print(coll.count())
print(nobel_winners) # mongodb has inserted ObjectIds in our original dict.

3
[{u'category': u'Physics', u'name': u'Albert Einstein', u'sex': u'male', u'year': 1921, u'nationality': u'Swiss', '_id': ObjectId('599d160fa2c4bb1f7401dc8b')}, {u'category': u'Physics', u'name': u'Paul Dirac', u'sex': u'male', u'year': 1933, u'nationality': u'British', '_id': ObjectId('599d160fa2c4bb1f7401dc8c')}, {u'category': u'Chemistry', u'name': u'Marie Curie', u'sex': u'female', u'year': 1911, u'nationality': u'Polish', '_id': ObjectId('599d160fa2c4bb1f7401dc8d')}]


In [43]:
# query/find items
res = coll.find({'category':'Chemistry'})
print('#1: \n{}'.format( list(res) ))

res = coll.find({'year': {'$gt': 1930}})
print('#2: \n{}'.format( list(res) ))

res = coll.find({'$or': [{'year': {'$gt': 1930}}, {'sex': 'female'}]})
print('#3: \n{}'.format( list(res) ))


#1: 
[{u'category': u'Chemistry', u'name': u'Marie Curie', u'sex': u'female', u'year': 1911, u'nationality': u'Polish', u'_id': ObjectId('599d160fa2c4bb1f7401dc8d')}]
#2: 
[{u'category': u'Physics', u'name': u'Paul Dirac', u'sex': u'male', u'year': 1933, u'nationality': u'British', u'_id': ObjectId('599d160fa2c4bb1f7401dc8c')}]
#3: 
[{u'category': u'Physics', u'name': u'Paul Dirac', u'sex': u'male', u'year': 1933, u'nationality': u'British', u'_id': ObjectId('599d160fa2c4bb1f7401dc8c')}, {u'category': u'Chemistry', u'name': u'Marie Curie', u'sex': u'female', u'year': 1911, u'nationality': u'Polish', u'_id': ObjectId('599d160fa2c4bb1f7401dc8d')}]


In [44]:
# recreate the original dict

def mongo_coll_to_dicts(dbname='test', collname='test', query=None, del_id=True, **kw):
    if not query: 
        query = {}
    db = get_mongo_database(dbname, **kw)
    res = list(db[collname].find(query))
    
    if del_id:
        for r in res:
            r.pop('_id')
            
    return res

new_nobel_winners = mongo_coll_to_dicts(DB_NOBEL_PRIZE, COLL_WINNERS)
print(new_nobel_winners)

[{u'category': u'Physics', u'name': u'Albert Einstein', u'sex': u'male', u'year': 1921, u'nationality': u'Swiss'}, {u'category': u'Physics', u'name': u'Paul Dirac', u'sex': u'male', u'year': 1933, u'nationality': u'British'}, {u'category': u'Chemistry', u'name': u'Marie Curie', u'sex': u'female', u'year': 1911, u'nationality': u'Polish'}]


In [38]:
# delete items
coll.delete_many({'category':'Chemistry'}) #specific items that meet filter
coll.delete_many({})  # deletes everything (no filter)
coll.count()

0